# Startup Lending Model (Project 2)

Insert a description of the model

- [**Setup**](#Setup): Runs any imports and other setup
- [**Inputs**](#Inputs): Defines the inputs for the model

## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [2]:
from dataclasses import dataclass
import pandas as pd
%matplotlib inline
import numpy_financial as npf

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [3]:
@dataclass
class ModelInputs:
    price_machine: float = 1000000
    loan_life: tuple = (5, 10, 20)
    initial_default: tuple = (0.1, 0.2, 0.3)
    default_decay: float = 0.9
    final_default: float = 0.4
    recovery_rate: float = 0.4
    interest_rates: tuple = (0.30, 0.35, 0.40)
    num_iterations: int = 1000
    case_names: tuple = ('Low Default', 'Base Case', 'High Default')
        
model_data = ModelInputs()
model_data

data = ModelInputs()

## Default Probability

In [6]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List
import numpy_financial as npf

@dataclass
class ModelInputs:
    """
    Model inputs with internal randomness integrated through tuples
    """
    price_machine: int = 1000000
    loan_life: tuple = (5, 10, 20)  # Different loan terms to evaluate
    initial_default: tuple = (0.1, 0.2, 0.3)  # p_default_1 scenarios
    default_decay: float = 0.9  # Decay_default
    final_default: float = 0.4  # p_default_n
    recovery_rate: float = 0.4  # r_recovery
    interest_rates: tuple = (0.30, 0.35, 0.40)  # Interest rates to evaluate (30%, 35%, 40%)

def calculate_default_probabilities(initial_prob: float, decay: float, final_prob: float, life: int) -> List[float]:
    """Calculate default probabilities for each year of the loan"""
    probs = []
    current_prob = initial_prob
    
    for year in range(1, life + 1):
        if year == life:  # Final year has special probability
            probs.append(final_prob)
        else:
            probs.append(current_prob)
            current_prob = current_prob * decay
    
    return probs

def calculate_expected_irr(price_machine: int, life: int, interest_rate: float, 
                         default_probs: List[float], recovery_rate: float) -> float:
    """Calculate expected IRR using analytical approach"""
    
    # Calculate cumulative survival probabilities
    survival_probs = [1.0]  # Start with 100% survival at time 0
    for i, default_prob in enumerate(default_probs):
        if i == 0:
            survival_probs.append(1 - default_prob)
        else:
            survival_probs.append(survival_probs[-1] * (1 - default_prob))
    
    scenarios = []
    
    # No default scenario
    no_default_prob = survival_probs[-1]
    no_default_cash_flows = [-price_machine]  # Initial investment
    # Regular interest payments
    for year in range(1, life):
        no_default_cash_flows.append(price_machine * interest_rate)
    # Final payment: principal + interest
    no_default_cash_flows.append(price_machine * (1 + interest_rate))
    
    no_default_irr = npf.irr(no_default_cash_flows)
    scenarios.append((no_default_prob, no_default_irr))
    
    # Default scenarios for each year
    for default_year in range(1, life + 1):
        # Probability of defaulting in this specific year
        if default_year == 1:
            default_year_prob = default_probs[0]
        else:
            # Survive to previous year * default in this year
            default_year_prob = survival_probs[default_year - 1] * default_probs[default_year - 1]
        
        # Build cash flows for default scenario
        cash_flows = [-price_machine]  # Initial investment
        
        # Regular payments before default
        for year in range(1, default_year):
            cash_flows.append(price_machine * interest_rate)
        
        # Default year and year after: no payments
        cash_flows.extend([0, 0])
        
        # Recovery payment 2 years after default
        cash_flows.append(recovery_rate * price_machine)
        
        default_irr = npf.irr(cash_flows)
        scenarios.append((default_year_prob, default_irr))
    
    # Calculate expected IRR
    expected_irr = sum(prob * irr for prob, irr in scenarios if not np.isnan(irr))
    
    return expected_irr

def run_analysis(inputs: ModelInputs) -> pd.DataFrame:
    """Run the complete analysis and return DataFrame with required columns"""
    results = []
    
    for interest_rate in inputs.interest_rates:
        for life in inputs.loan_life:
            for initial_default in inputs.initial_default:
                
                # Calculate default probabilities for this scenario
                default_probs = calculate_default_probabilities(
                    initial_default, inputs.default_decay, inputs.final_default, life
                )
                
                # Calculate expected IRR
                expected_irr = calculate_expected_irr(
                    inputs.price_machine, life, interest_rate, default_probs, inputs.recovery_rate
                )
                
                results.append({
                    'Interest Rate': interest_rate,
                    'Loan Life': life,
                    'Initial Default Probability': initial_default,
                    'IRR': expected_irr
                })
    
    return pd.DataFrame(results)

def verify_specific_scenarios(inputs: ModelInputs):
    """Verify against expected results from document"""
    print("Verification Scenarios (5 years, 20% interest, 20% initial default):")
    print("-" * 60)
    
    life = 5
    interest_rate = 0.20
    initial_default = 0.2
    price_machine = inputs.price_machine
    recovery_rate = inputs.recovery_rate
    
    # No default
    cash_flows = [-price_machine] + [price_machine * interest_rate] * (life - 1)
    cash_flows.append(price_machine * (1 + interest_rate))
    irr_no_default = npf.irr(cash_flows)
    print(f"No Default: {irr_no_default:.2%}")
    
    # Default scenarios
    for default_year in range(1, life + 1):
        cash_flows = [-price_machine]
        
        # Regular payments before default
        for year in range(1, default_year):
            cash_flows.append(price_machine * interest_rate)
        
        # Default and recovery
        cash_flows.extend([0, 0, recovery_rate * price_machine])
        
        irr = npf.irr(cash_flows)
        print(f"Default in t = {default_year}: {irr:.2%}")

# Main execution
def main():
    print("Probabilistic Loan Pricing Analysis")
    print("=" * 50)
    
    inputs = ModelInputs()
    
    # Show verification scenarios first
    verify_specific_scenarios(inputs)
    print("\n" + "=" * 50)
    
    # Run main analysis
    print("Running main analysis...")
    irr_df = run_analysis(inputs)
    
    # Format the output for better display
    irr_df_display = irr_df.copy()
    irr_df_display['Interest Rate'] = irr_df_display['Interest Rate'].apply(lambda x: f"{x:.0%}")
    irr_df_display['Initial Default Probability'] = irr_df_display['Initial Default Probability'].apply(lambda x: f"{x:.0%}")
    irr_df_display['IRR'] = irr_df_display['IRR'].apply(lambda x: f"{x:.2%}")
    
    print("\nMain Results:")
    print(irr_df_display.to_string(index=False))
    
    # Also show summary by interest rate
    print("\n" + "=" * 50)
    print("Summary by Interest Rate:")
    summary = irr_df.groupby('Interest Rate')['IRR'].agg(['mean', 'min', 'max'])
    summary_display = summary.copy()
    summary_display = summary_display.applymap(lambda x: f"{x:.2%}")
    summary_display.index = [f"{x:.0%}" for x in summary.index]
    print(summary_display)
    
    return irr_df

# Execute the analysis
irr_df = main()

Probabilistic Loan Pricing Analysis
Verification Scenarios (5 years, 20% interest, 20% initial default):
------------------------------------------------------------
No Default: 20.00%
Default in t = 1: -26.32%
Default in t = 2: -14.96%
Default in t = 3: -6.43%
Default in t = 4: 0.00%
Default in t = 5: 4.82%

Running main analysis...

Main Results:
Interest Rate  Loan Life Initial Default Probability    IRR
          30%          5                         10% 13.72%
          30%          5                         20%  5.08%
          30%          5                         30% -2.08%
          30%         10                         10% 15.68%
          30%         10                         20%  5.59%
          30%         10                         30% -2.20%
          30%         20                         10% 16.09%
          30%         20                         20%  5.73%
          30%         20                         30% -2.16%
          35%          5                         

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8336\2610379975.py:177: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary_display = summary_display.applymap(lambda x: f"{x:.2%}")
